In [29]:
import re
import spacy
from spacy.matcher import Matcher
import fitz
import PyPDF2

In [30]:
# Descargar el modelo si no está instalado
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    from spacy.cli import download
    download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")
    

In [31]:
def extraer_info(info_pdf):
    """Extraer los documento o proyectos que contiene la gaceta"""
    metameta = {}
    
    try:
        with fitz.open(info_pdf) as doc:
            texto_completo = ""
            for pagina in doc:
                texto_completo += pagina.get_text()
            
            texto_completo = preprocesar_texto(texto_completo)

            # 1. Nombre Principal
            if re.search(r"(?i)GACETA\s+DEL\s+CONGRESO", texto_completo):
                metameta["nombre"] = "Gaceta del Congreso"

            # --- ISSN ---
            metameta["issn"] = extraer_con_regex(r"I\s*S\s*S\s*N\s*(0\s*1\s*2\s*3\s*-\s*9\s*0\s*6\s*6)", texto_completo)
            if metameta["issn"]:
                metameta["issn"] = metameta["issn"].replace(" ", "")

            # --- 2. Fecha ---
            patron_fecha = r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})"
            coincidencia_fecha = re.search(patron_fecha, texto_completo)
            if coincidencia_fecha:
                meses = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
                         "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12}
                dia = int(coincidencia_fecha.group(2))
                mes = meses[coincidencia_fecha.group(3).lower()]
                anio = int(coincidencia_fecha.group(4))
                metameta["fecha"] = f"{dia:02d}/{mes:02d}/{anio}"

            # 3. Entidad (Senado/Cámara)
            match_entity = re.search(r"(?i)(SENADO Y CÁMARA|SENADO|CÁMARA)", texto_completo)
            if match_entity:
                entity = match_entity.group(1).strip()
                if entity == "SENADO Y CÁMARA":
                    metameta["entidades"] = ["SENADO Y CAMARA", "SENADO", "CÁMARA"]
                else:
                    metameta["entidades"] = [entity]

            # --- 4. Año (Números Romanos) y Número de Publicación ---
            patron_anio_numero = r"AÑO\s+([MDCLXVI]+)\s*-\s*N[°º]\s*(\d+)"
            coincidencia_anio_numero = re.search(patron_anio_numero, texto_completo, re.IGNORECASE)
            if coincidencia_anio_numero:
                anio_romano = coincidencia_anio_numero.group(1).strip()
                numero_publicacion = coincidencia_anio_numero.group(2).strip()

                metameta["anio_romano"] = anio_romano
                metameta["anio"] = romano_a_entero(anio_romano)
                metameta["numero_publicacion"] = int(numero_publicacion)

             # --- 6. Directores (Enfoque Híbrido Robusto) ---
            metameta["directores"] = {"senado": {}, "camara": {}}
            patron_directores_inicio = r"DIRECTORES:"
            coincidencia_inicio = re.search(patron_directores_inicio, texto_completo, re.IGNORECASE)

            if coincidencia_inicio:
                inicio_seccion = coincidencia_inicio.end()

                # Encuentra el final de la sección de directores (más robusto)
                fin_seccion = -1
                lineas = texto_completo[inicio_seccion:].splitlines()
                for i, linea in enumerate(lineas):
                    if not re.match(r"^\s*[A-ZÁÉÍÓÚÑ\s.,]+\s*(SECRETARIO GENERAL DEL SENADO|SECRETARIO GENERAL DE LA CÁMARA|RAMA LEGISLATIVA)?\s*$", linea.upper()): #se agrega la ,
                        fin_seccion = inicio_seccion + sum(len(l) + 1 for l in lineas[:i])  # +1 por salto de línea
                        break

                if fin_seccion != -1:
                    texto_directores = texto_completo[inicio_seccion:fin_seccion]

                    lineas = texto_directores.splitlines()

                    # Inicializa variables para guardar nombres y cargos
                    nombre_senado = None
                    cargo_senado = None
                    nombre_camara = None
                    cargo_camara = None

                    for i in range(len(lineas)):
                        linea = lineas[i].strip()
                        if not linea: #ignorar lineas vacías
                            continue

                        if nombre_senado is None and "SECRETARIO GENERAL DEL SENADO" not in linea.upper():
                            nombre_senado = linea #se obtiene el nombre del senado
                        elif cargo_senado is None and "SECRETARIO GENERAL DEL SENADO" in linea.upper():
                            cargo_senado = "SECRETARIO GENERAL DEL SENADO" #cargo del senado
                        elif nombre_camara is None and "SECRETARIO GENERAL DE LA CÁMARA" not in linea.upper() and cargo_senado is not None: #se valida el cargo senado
                            nombre_camara = linea #se obtiene el nombre de la camara
                        elif cargo_camara is None and "SECRETARIO GENERAL DE LA CÁMARA" in linea.upper():
                            cargo_camara = "SECRETARIO GENERAL DE LA CÁMARA" #se obtiene el cargo camara

                    if nombre_senado and cargo_senado:
                        metameta["directores"]["senado"]["nombre"] = nombre_senado
                        metameta["directores"]["senado"]["cargo"] = cargo_senado
                    if nombre_camara and cargo_camara:
                        metameta["directores"]["camara"]["nombre"] = nombre_camara
                        metameta["directores"]["camara"]["cargo"] = cargo_camara
                else:
                    print("No se pudo encontrar el final de la sección de directores.")

                if fin_seccion != -1:
                    texto_directores = texto_completo[inicio_seccion:fin_seccion]
                    
                    lineas = texto_directores.splitlines()
                    nombre_senado = None
                    cargo_senado = None
                    nombre_camara = None
                    cargo_camara = None

                if texto_directores:
                    doc_directores = nlp(texto_directores)

                    for ent in doc_directores.ents: # dentro del if
                        if ent.label_ == "PER":
                            nombre = ent.text.strip()
                            cargo = None 
                        
                            for sent in doc_directores.sents:
                                if nombre in sent.text:
                                    for token in sent:
                                        if "SECRETARIO" in token.text.upper() and "GENERAL" in token.text.upper():
                                            cargo = token.text
                                            if "SENADO" in cargo.upper():
                                                metameta["directores"]["senado"] = {"nombre": nombre, "cargo": cargo}
                                            elif "CÁMARA" in cargo.upper():
                                                metameta["directores"]["camara"] = {"nombre": nombre, "cargo": cargo}
                                            break 
                                    if cargo:
                                        break

                                
                                
            # --- Segundo grupo Documentos (REGEX + Spac) ----
            documentos = []
            pattern_tipo_documento = r"(?i)(PROYECTO DE LEY|PROYECTO DE LEY ESTATUTARIA|PROYECTO DE ACTO LEGISLATIVO|PROYECTO DE RESOLUCIÓN|PROYECTO DE ACUERDO|PROYECTO DE ORDENANZA|PROYECTO DE DECRETO|PROYECTO DE RESOLUCIÓN|PROYECTO DE ACUERDO|PROYECTO DE ORDENANZA|PROYECTO DE DECRETO)"
            
            for match in re.findeiter(pattern_tipo_documento, texto_completo):
                tipo_documento = match.group(1).strip
                texto_restante = match.group(2)
                
                doc_spacy = nlp(texto_restante)
                
                subtitulo = ""
                subsubtitulo = ""
                
                #extraer subtitulo (primera linea)
                for sent in doc_spacy.sents:
                    subtitulo = sent.text.strip()
                    if subtitulo:
                        break
                
                # Extraer subtitulo
                if subtitulo:
                    text_after_subtitle = texto_restante[texto_restante.find(subtitulo) + len(subtitulo):].strip()
                    
                    # search into   
                    match_subsubtitulo  = re.search(r'"(.*?)"', text_after_subtitle)
                    if match_subsubtitulo:
                        subsubtitulo = match_subsubtitulo.group(1)
                        
                    else: #si no se encuentra entre comillas, buscar la siguiente frase
                         for sent in nlp(text_after_subtitle).sents:
                            posible_subsubtitulo = sent.text.strip()
                            if posible_subsubtitulo:
                                subsubtitulo = posible_subsubtitulo
                                break
                            
                documentos.append({
                    "tipo_documento": tipo_documento,
                    "subtitulo": subtitulo,
                    "subsubtitulo": subsubtitulo
                })
                
            metameta["documentos"] = documentos

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo PDF: {info_pdf}")
        return None
    except Exception as e:
        print(f"Error al procesar el PDF {info_pdf}: {e}")
        return None
            
    return metameta
        

In [32]:
def extraer_con_regex(pattern, text):
    coincidencia = re.search(pattern, text)
    if coincidencia:
        return coincidencia.group(1)
    return None

def preprocesar_texto(texto):
    texto_limpio = re.sub(r"\s+", " ", texto).strip()  # Elimina espacios extra
    texto_limpio = texto_limpio.replace('\n', ' ')  # elimina saltos de linea
    return texto_limpio

def leer_pdf(ruta_pdf):
    """Lee un archivo PDF y devuelve su contenido en texto."""
    documento = fitz.open(ruta_pdf)
    texto = ""
    for pagina in documento:
        texto += pagina.get_text()
    return texto

def separar_documento(text):
    pattern_tipo_documento = r"(?i)(PONENCIA|ACTA|PROYECTO DE LEY|INFORME|RESOLUCIÓN|CONCEPTO|PROPOSICIÓN|CONSTANCIA|OBJECIONES|CONCEPTOS JURÍDICOS|LEYES SANCIONADAS|PRESENTACIÓN)(.*)"
    documentos = re.findall(pattern_tipo_documento, text)
    documentos = [doc[1].strip() for doc in documentos if doc[1].strip()]
    
    return documentos

In [33]:

def romano_a_entero(romano):
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
               'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero


In [34]:
ruta_gaceta = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64.pdf"  # Reemplaza con la ruta correcta
datos_extraidos = extraer_info(ruta_gaceta) # Pasar la ruta directamente

if datos_extraidos:
    print(datos_extraidos)

Error al procesar el PDF C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64.pdf: module 're' has no attribute 'findeiter'


In [35]:
# # --- Ejemplo de Uso ---
# ruta_gaceta = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328 XXV 110_64.pdf"  # Reemplaza con la ruta de tu PDF
# texto_prueba = leer_pdf(ruta_gaceta)
# datos_extraidos = extraer_info(texto_prueba)

# if datos_extraidos:
#     print(datos_extraidos)